In [5]:
from PIL import Image
import ipywidgets as widgets
from IPython.display import clear_output
from ipywidgets import interact, fixed, IntSlider

In [2]:
@interact(color=widgets.Checkbox(value=True, description="Add Colour", disabled=False, continuous_update=True))
def convert_image(color):
    img = Image.open("./test_image.png")
    print(color)
    # clear_output(wait=True)
    return img if color else img.convert("L") 
    # display(result)

interactive(children=(Checkbox(value=True, description='Add Colour'), Output()), _dom_classes=('widget-interac…

In [3]:
def my_function(a, b):
    return a + b

interact(my_function, a=5, b=10)

interactive(children=(IntSlider(value=5, description='a', max=15, min=-5), IntSlider(value=10, description='b'…

<function __main__.my_function(a, b)>

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO
import torch
from torchvision import transforms
from matplotlib import cm

# Define a few simple noise schedulers
class LinearScheduler:
    def __init__(self, beta_start=0.0001, beta_end=0.02, timesteps=1000):
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.timesteps = timesteps
        
        # Linear schedule
        self.betas = torch.linspace(beta_start, beta_end, timesteps)
        self.alphas = 1.0 - self.betas
        self.alphas_cumprod = torch.cumprod(self.alphas, dim=0)
        
    def add_noise(self, img, t):
        """Add noise to image at timestep t"""
        if isinstance(t, int):
            t = torch.tensor([t])
        
        sqrt_alpha_cumprod = torch.sqrt(self.alphas_cumprod[t])
        sqrt_one_minus_alpha_cumprod = torch.sqrt(1 - self.alphas_cumprod[t])
        
        noise = torch.randn_like(img)
        noisy_img = sqrt_alpha_cumprod * img + sqrt_one_minus_alpha_cumprod * noise
        return noisy_img, noise

class CosineScheduler:
    def __init__(self, timesteps=1000, s=0.008):
        self.timesteps = timesteps
        
        # Cosine schedule as proposed in https://arxiv.org/abs/2102.09672
        steps = timesteps + 1
        x = torch.linspace(0, timesteps, steps)
        alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s) * torch.pi * 0.5) ** 2
        alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
        betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
        betas = torch.clip(betas, 0.0001, 0.9999)
        
        self.betas = betas
        self.alphas = 1.0 - betas
        self.alphas_cumprod = torch.cumprod(self.alphas, dim=0)
    
    def add_noise(self, img, t):
        """Add noise to image at timestep t"""
        if isinstance(t, int):
            t = torch.tensor([t])
        
        sqrt_alpha_cumprod = torch.sqrt(self.alphas_cumprod[t])
        sqrt_one_minus_alpha_cumprod = torch.sqrt(1 - self.alphas_cumprod[t])
        
        noise = torch.randn_like(img)
        noisy_img = sqrt_alpha_cumprod * img + sqrt_one_minus_alpha_cumprod * noise
        return noisy_img, noise

class QuadraticScheduler:
    def __init__(self, beta_start=0.0001, beta_end=0.02, timesteps=1000):
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.timesteps = timesteps
        
        # Quadratic schedule
        self.betas = torch.linspace(0, 1, timesteps) ** 2 * (beta_end - beta_start) + beta_start
        self.alphas = 1.0 - self.betas
        self.alphas_cumprod = torch.cumprod(self.alphas, dim=0)
    
    def add_noise(self, img, t):
        """Add noise to image at timestep t"""
        if isinstance(t, int):
            t = torch.tensor([t])
        
        sqrt_alpha_cumprod = torch.sqrt(self.alphas_cumprod[t])
        sqrt_one_minus_alpha_cumprod = torch.sqrt(1 - self.alphas_cumprod[t])
        
        noise = torch.randn_like(img)
        noisy_img = sqrt_alpha_cumprod * img + sqrt_one_minus_alpha_cumprod * noise
        return noisy_img, noise

# Load a sample image
def load_sample_image():
    # Either load an image from a URL
    img = Image.open("./test_image.png")
    
    # Transform to tensor
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),  # Scales to [0, 1]
        transforms.Lambda(lambda x: x * 2 - 1)  # Scale to [-1, 1]
    ])
    
    return transform(img).unsqueeze(0)

# Create the schedulers
linear_scheduler = LinearScheduler()
cosine_scheduler = CosineScheduler()
quadratic_scheduler = QuadraticScheduler()

# Load the sample image
sample_img = load_sample_image()

# Create the visualization
def visualize_noise(t=0):
    t = int(t)
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Set a consistent random seed for each visualization call
    torch.manual_seed(42)
    
    # Add noise to the image with each scheduler
    noisy_linear, _ = linear_scheduler.add_noise(sample_img, t)
    
    torch.manual_seed(42)
    noisy_cosine, _ = cosine_scheduler.add_noise(sample_img, t)
    
    torch.manual_seed(42)
    noisy_quadratic, _ = quadratic_scheduler.add_noise(sample_img, t)
    
    # Function to denormalize and convert to numpy for display
    def process_for_display(img_tensor):
        img = img_tensor.squeeze(0).permute(1, 2, 0).detach().numpy()
        img = (img + 1) / 2  # Scale from [-1, 1] to [0, 1]
        img = np.clip(img, 0, 1)
        return img
    
    # Display images
    axes[0].imshow(process_for_display(noisy_linear))
    axes[0].set_title(f'Linear Scheduler (t={t})')
    axes[0].axis('off')
        
    axes[1].imshow(process_for_display(noisy_quadratic))
    axes[1].set_title(f'Quadratic Scheduler (t={t})')
    axes[1].axis('off')

    axes[2].imshow(process_for_display(noisy_cosine))
    axes[2].set_title(f'Cosine Scheduler (t={t})')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Display plots of noise schedules
    fig, ax = plt.subplots(figsize=(10, 4))
    timesteps = np.arange(0, 1000)
    ax.plot(timesteps, linear_scheduler.alphas_cumprod.numpy(), label='Linear α_cumprod')
    ax.plot(timesteps, cosine_scheduler.alphas_cumprod.numpy(), label='Cosine α_cumprod')
    ax.plot(timesteps, quadratic_scheduler.alphas_cumprod.numpy(), label='Quadratic α_cumprod')
    
    # Highlight current position
    ax.axvline(x=t, color='r', linestyle='--')
    
    ax.set_xlabel('Timestep')
    ax.set_ylabel('Cumulative Product of Alphas')
    ax.set_title('Noise Schedule Comparison')
    ax.legend()
    plt.tight_layout()
    plt.show()

# Create interactive widget
interactive_plot = interact(visualize_noise, t=IntSlider(min=0, max=999, step=10, value=0, description='Timestep:'), continuous_update=False)
output = interactive_plot.widget.children[-1]
output.layout.height = '700px'
interactive_plot

interactive(children=(IntSlider(value=0, description='Timestep:', max=999, step=10), Output()), _dom_classes=(…

<function __main__.visualize_noise(t=0)>